# Lab 3: Feature Engineering and SVM Model

在本实验中，我们将聚焦数据特征工程
1. 数据预处理的基本步骤
2. 特征工程的常用技巧
3. 特征预筛选

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


## 1. Say hello to the old friend



In [2]:
data = pd.read_csv('../data/loan_data.csv').head(2000)  # 为了避免后面计算性能降低太多
X = data.drop('loan_status', axis=1)
y = data['loan_status']

# 原则上，你不应该知道test数据集的分布
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. 数据预处理

### 2.1 处理缺失值

大部分现实世界的数据集都有大量的缺失值。处理缺失值的不同方法可能会对于模型性能产生显著的影响。处理缺失值的方法

    - 删除：最简单但是最糟糕的做法。不仅导致数据量减少，而且实际上改变了数据集分布，增加了模型过拟合风险，降低泛化能力
    - 填充：
        - 均值填充/中位数填充/众数填充：简单易行，比删除数据好，但是也会影响模型效果。特别是缺失比例较高时
        - 模型填充：比较复杂。可能化腐朽为神奇，也可能garbage in garbage out。

在我们的lab和我推荐的大部分数据集里，我们都不需要考虑缺失值处理。

## 2.2 One-hot Encoding

数据集里的离散变量需要被提前识别和转化为一系列0/1变量。这个过程叫One hot encoding。

在One Hot Encoding中，一般都会drop一个类别，否则会因为变量间共线性导致模型无法收敛。这个对于线性模型是致命的，但是对于其他非线形模型并不重要。

In [9]:
from sklearn.preprocessing import OneHotEncoder

# 数值和分类特征列表

categorical_features = [
    'person_gender', 'person_education', 'person_home_ownership',
    'loan_intent', 'previous_loan_defaults_on_file'
]

transformers = [
    ('cat', OneHotEncoder(drop='first'), categorical_features)
]


*TODO*: 看看现在categorical_features里有哪些变量

### 2.2 特征缩放比较

下面我们来展示一个标准的特征缩放过程，即正态化。这避免某些特征的尺度对于其他特征的影响。比如把某个省的GDP和它的上年增长率放在一起，GDP的尺度会对于模型产生显著影响。

NOTE: copy的作用是保留原始数据，在反复运行中不会污染原始数据

In [10]:
import copy

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron

numerical_features = [
    'person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
    'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
    'credit_score'
]
transformers_copy = copy.deepcopy(transformers)
transformers_copy.append(('num', StandardScaler(), numerical_features))

preprocessor = ColumnTransformer(transformers_copy)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', Perceptron(random_state=42))
])




*TODO*：绘制原始数据的mean和std，以及标准化后的mean和std

In [ ]:
# 在完整训练集上训练最终模型
pipeline.fit(X_train, y_train)


# 在测试集上评估模型性能
y_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("使用Pipeline的分类报告:\n", classification_report(y_test, y_pred))

*HW* 为什么predict时没有做特征处理？what happens？pipeline起了什么作用？

**正态化不是唯一的缩放方法**

原则上processors的编排也是可以用cross validation来进行筛选的

In [ ]:
from sklearn.model_selection import cross_val_score
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# 用cross validation选择最佳的scaler
cv_results = {}

for name, scaler in scalers.items():
    transformers_copy = copy.deepcopy(transformers)
    transformers_copy.append(('num', scaler, numerical_features))

    # 创建pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(transformers_copy)),
        ('classifier', Perceptron(random_state=42))
    ])
    
    # 使用5折交叉验证评估模型
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 存储结果
    cv_results[name] = scores.mean()

# 打印每个scaler的平均准确率
for name, score in cv_results.items():
    print(f"{name} 的平均交叉验证准确率: {score:.6f}")

# 找出最佳的scaler
best_scaler = max(cv_results.items(), key=lambda x: x[1])[0]
print(f"\n最佳scaler是: {best_scaler}")





## 3. 特征工程

最常见也一般最有效的特征工程方法是特征交互。

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# 使用最佳的scaler创建预处理器
transformers_copy = copy.deepcopy(transformers)
transformers_copy.append(('num',  StandardScaler(), numerical_features))

preprocessor = ColumnTransformer(transformers_copy)

# 对训练集和测试集进行转换
X_train_scaled = preprocessor.fit_transform(X_train)
# 创建多项式特征
poly = PolynomialFeatures(degree=2, interaction_only=True) 
X_train_poly = poly.fit_transform(X_train_scaled)

print("原始特征数量:", X_train.shape[1])
print("交互特征后的数量:", X_train_poly.shape[1])

**HW** 注意上面用了interactions only这个参数。试试去掉这个参数会发生什么？为什么会出现这个现象？

将这个流程添加到pipeline中

In [ ]:
# 创建完整的pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
    ('classifier', Perceptron(random_state=42))
])

# 在训练集上拟合pipeline
pipeline.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = pipeline.predict(X_test)

# 打印分类报告
print("使用Pipeline的分类报告:\n", classification_report(y_test, y_pred))


*TODO*: 比较两个pipeline的结果，这说明了什么？让大模型帮助你写一个能被其他人读懂的“学术描述”。这是论文写作的核心

## 4. SVM & Kernel SVM

## 4.1 SVM参数调优

首先，让我们调整C参数。回忆lab 2的参数调整方法

In [ ]:
param_c_list = [0.1, 1, 10, 100]
param_grid = {
    'classifier__C': param_c_list,  # 添加classifier__前缀
}

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
    ('classifier', SVC(random_state=42, kernel='linear'))
])

# 创建网格搜索对象
grid_search = GridSearchCV(
    pipeline,  # 使用pipeline作为estimator
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=4
)

# 在训练集上进行网格搜索
grid_search.fit(X_train, y_train)  

print("最佳参数:", grid_search.best_params_)
print("最佳得分:", grid_search.best_score_)

*TODO* 把n_jobs设置为4，比较运行时间；

In [ ]:
# 获取最佳模型
best_model = grid_search.best_estimator_

# 获取预处理后的特征名称
feature_names = (
    # 获取分类特征的名称
    best_model.named_steps['preprocessor']
    .named_transformers_['cat']
    .get_feature_names_out([
        'person_gender', 'person_education', 'person_home_ownership',
        'loan_intent', 'previous_loan_defaults_on_file'
    ]).tolist() +
    # 获取数值特征的名称
    [
        'person_age', 'person_income', 'person_emp_exp',
        'loan_amnt', 'loan_int_rate', 'loan_percent_income',
        'cb_person_cred_hist_length', 'credit_score'
    ]
)

# 获取多项式特征的名称
poly_features = best_model.named_steps['poly'].get_feature_names_out(feature_names)

# 获取SVC的系数（仅适用于线性核）
if best_model.named_steps['classifier'].kernel == 'linear':
    coefficients = best_model.named_steps['classifier'].coef_[0]
    
    # 创建特征名称和系数的DataFrame
    coef_df = pd.DataFrame({
        '特征': poly_features,
        '系数': coefficients
    })
    
    # 显示系数为0的特征
    print("系数为0的特征：")
    print(coef_df[coef_df['系数'] == 0])
    
    # 显示绝对值最大的前10个系数
    print("\n影响最大的前10个特征：")
    print(coef_df.reindex(coef_df['系数'].abs().sort_values(ascending=False).index)[:10])
else:
    print("非线性核函数不能直接解释特征重要性")

## 4.2 核函数
核函数是另一种形式的“特征”工程。它将特征从欧几里得空间映射到其他（高维）空间，从而改变了“距离”的物理含义。

虽然早期核函数的应用都有一定的现实原因，但是随着kernel trick的提出，核函数逐渐成为了一种数学技巧，而不再需要现实原因。

广义地说，核函数描述了数据生成过程（data generating process, DGP）的一部分，policy的目标是尽可能拟合DGP。

In [ ]:
# 定义新的参数网格,增加kernel的搜索
param_grid = {
    'classifier__C': param_c_list,
    'classifier__kernel': ['linear', 'rbf'],
}

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
    ('classifier', SVC(random_state=42))
])

# 使用网格搜索
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=4
)

# 训练模型
grid_search.fit(X_train, y_train)

# 打印最佳参数
print("最佳参数:", grid_search.best_params_)

# 在测试集上评估
y_pred = grid_search.predict(X_test)
print("\n测试集准确率:", accuracy_score(y_test, y_pred))
print("\n分类报告:")
print(classification_report(y_test, y_pred))


## 5. 预训练阶段的特征选择

事实上即使我们不去拟合复杂模型，也可以对于特征进行有效选择，这样可以降低计算成本。

*TODO/HW* 尝试使用logistic regression 对特征进行预选择，将精炼后的特征再放进SVM中

In [29]:
# Pair program with LLM here

# 6.总结

这个lab结束后，我们掌握了5种工具来改善模型最后的表现：
- 编码（onehot encoding）和缩放（Scaler）
- 特征工程【我们只讲了多项式这一种技巧，事实上还有其他技巧】
- 模型
- 模型参数的Grid Search
- 评估指标

还有2类工具我们没有讲但是在实际分类任务中非常重要
- 缺失值处理
- 样本不均衡处理 （例如正样本只有2%要怎么办？）


因此，ML的最终调优是 算法技术，算力和分析经验共同作用的结果。经过这么多年发展，现在约束ML调优的基本上就是算力和分析经验驱动的特征工程了


就在Data Scientist成为Data Monkey的时候，出现了深度学习革命。